In [1]:
#imports
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import re
import time
import random
import sys
pd.set_option('display.max_colwidth', None)

from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity
from scipy import sparse
from matplotlib import pyplot as plt
import utils as ut

In [60]:
def rec_search(category, query, wout='no'):

    if category.lower() == 'video games':
        lookup, recommender = pd.read_pickle('./pickles/videog_look.pkl'),pd.read_pickle('./pickles/videog_rec.pkl')
    elif category.lower() == 'movies':
        lookup, recommender = pd.read_pickle('./pickles/movies_look.pkl'), pd.read_pickle('./pickles/movie_rec.pkl') 
    elif category.lower() == 'books':
        lookup, recommender = pd.read_pickle('./pickles/books_look.pkl'), pd.read_pickle('./pickles/books_rec.pkl')
    else:
        return "Sorry, that wasn't one of the available categories"

    try:
        query=query.title() #uppercase first letter of each word in the query in case it's not entered that way
        titles = list(lookup[lookup['product_title'].str.contains(query)]['product_title'])
        q = titles[0] #this is the item to search for

        if wout.lower() == 'no':
            query_dict = dict(recommender.loc[q].sort_values())
            
            final_printout = ''
            for key in list(query_dict.keys())[1:11]:
                final_printout += '\n' #add new line
                final_printout += f'{key}' #print item name
                final_printout += '\n'
                final_printout += f"""
                This item has {round(lookup[lookup['product_title']==key]['tot_prod_reviews'].mean())} reviews 
                and a {round(lookup[lookup['product_title']==key]['avg_prod_stars'].mean(), 2)} average star rating"""
                final_printout += '\n'

            return print(f'''

            Recommending items similar to: {q}
            This item has {round(lookup[lookup['product_title']==q]['tot_prod_reviews'].mean())} reviews
            and a {round(lookup[lookup['product_title']==q]['avg_prod_stars'].mean(), 2)} average star rating.

            Here are the 10 recommended items for you based on your search parameters:
            
            {final_printout}
            
            ''')
            
        else:
            
            wout = wout.title() #capitlize first letters
            query_dict = dict(recommender.loc[q].sort_values())
            filtered_query = [] #make empty list
            for key, value in query_dict.items(): #index into dictionary of results
                if wout not in key: #check if avoided keyword is in results
                    filtered_query.append((key, value)) #make list of results that DON'T include "wout" keyword
            
            final_printout = ''
            for item in filtered_query[1:11]:
                final_printout += '\n' #add new line
                final_printout += f'{item[0]}' #print item name
                final_printout += '\n'
                final_printout += f"""
                This item has {round(lookup[lookup['product_title']==item[0]]['tot_prod_reviews'].mean())} reviews 
                and a {round(lookup[lookup['product_title']==item[0]]['avg_prod_stars'].mean(), 2)} average star rating"""
                final_printout += '\n'
            return print(f'''

            Recommending items similar to: {q}
            This item has {round(lookup[lookup['product_title']==q]['tot_prod_reviews'].mean())} reviews
            and a {round(lookup[lookup['product_title']==q]['avg_prod_stars'].mean(), 2)} average star rating.

            Here are the 10 recommended items for you based on your search EXCLUDING "{wout}":
            
            {final_printout}
            
            ''')
        
    except:
        return print(f'Sorry, "{query}" does not appear to be in the product database')

In [61]:
rec_search('books', 'batman')



            Recommending items similar to: Batman: The Dark Knight Returns
            This item has 177 reviews
            and a 4.64 average star rating.

            Here are the 10 recommended items for you based on your search parameters:
            
            
Watchmen

                This item has 203 reviews 
                and a 4.68 average star rating

Batman: Year One

                This item has 39 reviews 
                and a 4.72 average star rating

Batman: The Killing Joke

                This item has 80 reviews 
                and a 4.51 average star rating

Batman: Arkham Asylum

                This item has 33 reviews 
                and a 3.82 average star rating

Batman: Year One (Batman (DC Comics Hardcover))

                This item has 22 reviews 
                and a 4.59 average star rating

V for Vendetta

                This item has 92 reviews 
                and a 4.54 average star rating

Batman: The Long Halloween

                

Okay - this works and I got it running my flask app! What if, however, I return my output as a DATAFRAME instead of a long text segment? I think the table-ish formatting would look better and be easier to read.

In [2]:
#read in pickled dataframes
books_df, books_rec, movies_df, movies_rec, vg_df, vg_rec = pd.read_pickle('./pickles/books_look.pkl'), pd.read_pickle('./pickles/books_rec.pkl'), pd.read_pickle('./pickles/movies_look.pkl'), pd.read_pickle('./pickles/movie_rec.pkl'), pd.read_pickle('./pickles/videog_look.pkl'), pd.read_pickle('./pickles/videog_rec.pkl')

In [32]:
def rec_search(category, query, wout='no'):

    if category.lower() == 'video games':
        lookup, recommender = vg_df, vg_rec #pd.read_pickle('./pickles/videog_look.pkl'),pd.read_pickle('./pickles/videog_rec.pkl')
    elif category.lower() == 'movies':
        lookup, recommender = movies_df, movies_rec #pd.read_pickle('./pickles/movies_look.pkl'), pd.read_pickle('./pickles/movie_rec.pkl') 
    elif category.lower() == 'books':
        lookup, recommender = books_df, books_rec #pd.read_pickle('./pickles/books_look.pkl'), pd.read_pickle('./pickles/books_rec.pkl')
    else:
        return "Sorry, that wasn't one of the available categories"

    try:
        query=query.title() #uppercase first letter of each word in the query in case it's not entered that way
        titles = list(lookup[lookup['product_title'].str.contains(query)]['product_title'])
        q = titles[0] #this is the item to search for

        if wout.lower() == 'no':
            query_dict = dict(recommender.loc[q].sort_values())
            
            top10_prods = []
            num_prod_revs = []
            avg_prod_stars = []
            for key in list(query_dict.keys())[1:11]:
                top10_prods.append(key)
                num_prod_revs.append(round(lookup[lookup['product_title']==key]['tot_prod_reviews'].mean()))
                avg_prod_stars.append(round(lookup[lookup['product_title']==key]['avg_prod_stars'].mean(), 2))
            #print(top10_prods, num_prod_revs, avg_prod_stars)
            final_output_df = pd.DataFrame(data = {
                'Recommended Items':top10_prods,
                'Total Reviews for Product':num_prod_revs,
                'Avg Product Star Rating(1-5)':avg_prod_stars
            }, index=range(1,11))

            return final_output_df            
            
            
            
        else:
            
            wout = wout.title() #capitlize first letters
            query_dict = dict(recommender.loc[q].sort_values())
            filtered_query = [] #make empty list
            for key, value in query_dict.items(): #index into dictionary of results
                if wout not in key: #check if avoided keyword is in results
                    filtered_query.append((key, value)) #make list of results that DON'T include "wout" keyword
            
            top10_prods = []
            num_prod_revs = []
            avg_prod_stars = []
            for item in filtered_query[1:11]:
                top10_prods.append(item[0])
                num_prod_revs.append(round(lookup[lookup['product_title']==item[0]]['tot_prod_reviews'].mean()))
                avg_prod_stars.append(round(lookup[lookup['product_title']==item[0]]['avg_prod_stars'].mean(), 2))
            
                
            final_output_df = pd.DataFrame(data = {
                'Recommended Items':top10_prods,
                'Total Reviews for Product':num_prod_revs,
                'Avg Product Star Rating(1-5)':avg_prod_stars
                }, index=range(1,11))
                
                
                
            return final_output_df
        
    except:
        return f'Sorry, "{query}" does not appear to be in the product database'

In [33]:
rec_search('books', 'batman', 'batman')

,Recommended Items,Total Reviews for Product,Avg Product Star Rating(1-5)
1,V for Vendetta,92,4.54
2,"Daredevil Visionaries - Frank Miller, Vol. 1",18,4.28
3,Marvels,31,4.48
4,Frank Miller's Ronin,24,4.38
5,Infinity Gauntlet TPB,12,4.58
6,New X-Men Vol. 1: E is for Extinction (v. 1),27,4.26
7,Kingdom Come,134,4.37
8,Marvel 1602 HC (Marvel Heroes),16,4.25
9,Preacher VOL 06: War in the Sun (Preacher (DC Comics)),15,4.93
10,"6: Sandman, The: Fables & Reflections - Book VI",19,4.32
